In [265]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
! pip install Pillow
! pip install pytesseract
from pytesseract import image_to_string
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm



In [226]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [270]:
def generate_text(image):
  text = image_to_string(image)
  return text

file_path = "/content/drive/My Drive/RC/"
img_dir = ['txt_mudit_b11_11599.jpg','txt_mudit_b11_12.jpg','txt_mudit_b11_1328.jpg','txt_mudit_b11_1330.jpg','txt_mudit_b11_1332.jpg',
           'txt_mudit_b11_1334.jpg','txt_mudit_b11_1337.jpg','txt_mudit_b11_1339.jpg','txt_mudit_b11_1343.jpg','txt_mudit_b11_1347.jpg',
           'txt_mudit_b11_1354.jpg','txt_mudit_b11_1355.jpg','txt_mudit_b11_1361.jpg','txt_mudit_b11_1362.jpg','txt_mudit_b11_1363.jpg',
           'txt_mudit_b11_1364.jpg','txt_mudit_b11_1365.jpg','txt_mudit_b11_1369.jpg','txt_mudit_b11_1524.jpg','txt_mudit_b11_1529.jpg',
           'txt_mudit_b11_1534.jpg','txt_mudit_b11_1535.jpg','txt_mudit_b11_1536.jpg','txt_mudit_b11_1539.jpg','txt_mudit_b11_439.jpg',
           'txt_mudit_b11_480.jpg','txt_mudit_b11_746.jpg','txt_mudit_b11_961.jpg','txt_mudit_b8_1_513.jpg','txt_mudit_b8_1_545.jpg',
           'txt_mudit_b8_1_546.jpg','txt_mudit_b8_1_554.jpg','txt_mudit_b8_1_555.jpg','txt_mudit_b8_1_571.jpg','txt_mudit_b8_1_573.jpg',
           'txt_mudit_b8_1_574.jpg','txt_mudit_b8_1_740.jpg','txt_mudit_b8_1_741.jpg','txt_mudit_b8_1_742.jpg','txt_mudit_b8_1_806.jpg',
           'txt_mudit_b8_1_806 (1).jpg','txt_mudit_b8_1_807.jpg','txt_mudit_b8_1_808.jpg','txt_mudit_b8_1_854 (1).jpg','txt_mudit_b8_1_854.jpg',
           'txt_mudit_b8_1_855 (1).jpg','txt_mudit_b8_1_855.jpg','txt_mudit_b8_2_213 (1).jpg','txt_mudit_b8_2_213 (1).jpg',]
# Lists for storing the information on pictures
Regn_ls=[]
Name_ls=[]
Eng_ls=[]
Mfg_ls=[]
Reg_ls=[]
Chas_ls=[]
Sr_no = 0
for p in tqdm(range(49)):



# Preparing mask to remove glares from images
  img_name = img_dir[p]
  mask = cv2.imread(file_path+img_name)

  for i in range(mask.shape[0]):
      for j in range(mask.shape[1]):
          if(mask[i,j,0]>250 or mask[i,j,1]>250 or mask[i,j,2]>250):
              mask[i,j,2]=0
              mask[i,j,1]=0
              mask[i,j,0]=0
          else:
              mask[i,j,2]=255
              mask[i,j,1]=255
              mask[i,j,0]=255
              
  for i in range(mask.shape[0]):
      for j in range(mask.shape[1]):
          if(mask[i,j,0]==255 or mask[i,j,1]==255 or mask[i,j,2]==255):
              mask[i,j,2]=0
              mask[i,j,1]=0
              mask[i,j,0]=0
          else:
              mask[i,j,2]=255
              mask[i,j,1]=255
              mask[i,j,0]=255

  img = cv2.imread(file_path+img_name)

# Removing the glares

  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)

  img2 = cv2.inpaint(img, mask, 15, cv2.INPAINT_NS)
# Applying sharpening kernel to do the convolution operation and remove blurs from the images
  sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
  img3 = cv2.filter2D(img2, -1, sharpen_kernel)


  img4 = img3.reshape((-1,3))

  img4 =  np.float32(img4)

  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

  attempts = 10
  # Using k means clustering pixel wise.
  k=20
  ret,label,center = cv2.kmeans(img4,k,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
  center = np.uint8(center)

  res = center[label.flatten()]
  res2 = res.reshape((img.shape))



  Sr_no= Sr_no +1
  # Using  image_to_string function via generate_text() function call to generate texts from images
  txt = generate_text(res2)
  

  # Storing the info
  ls=[]
  word =""
  for a in txt:
      if a is not "\n":
          word =word+a
      else:
          #print(word)
          ls.append(word)
          word=""

  for b in ls:
      if b.find("REGN")!=-1 or b.find("tion")!=-1:
          #print(b)
          Regn_no = b
          Regn_ls.append(str(Sr_no) +" "+ Regn_no)

  for c in ls:
      if c.find("NAME")!=-1 or c.find("name")!=-1 or c.find("Name")!=-1 or c.find("nam")!=-1 or c.find("Name")!=-1 or c.find("NAM")!=-1:
          #print(c)
          Name = c
          Name_ls.append(str(Sr_no) +" "+ Name)
          
  for d in ls:
      if d.find("ENO")!=-1 or d.find("Engine")!=-1:
          #print(d)
          Eng_no = d
          Eng_ls.append(str(Sr_no) +" "+ Eng_no)

  for e in ls:
      if e.find("REG. DT")!=-1 or e.find("Date of Issue")!=-1:
          #print(e)
          Reg_date = e
          Reg_ls.append(str(Sr_no) +" "+ Reg_date)
          
  for f in ls:
      if f.find("MFG-DT")!=-1 or f.find("MFG")!=-1 or f.find("MEG")!=-1:
          #print(f)
          Mfg_date = f
          Mfg_ls.append(str(Sr_no) +" "+ Mfg_date)
          
  for g in ls:
      if g.find("CH. NO")!=-1 or g.find("Chasis No.")!=-1 or g.find("asis")!=-1:
          #print(g)
          Chassis_no = g
          Chas_ls.append(str(Sr_no) +" "+ Chassis_no)
 
#cv2_imshow(res2)


100%|██████████| 49/49 [43:16<00:00, 107.06s/it]


In [0]:
# Saving the info as dataframes
df1 = pd.DataFrame(Regn_ls)
df2 = pd.DataFrame(Name_ls)
df3 = pd.DataFrame(Eng_ls)
df4 = pd.DataFrame(Mfg_ls)
df5 = pd.DataFrame(Reg_ls)
df6 = pd.DataFrame(Chas_ls)

dff = pd.concat([df1,df2,df3,df4,df5,df6],axis=1)
dff.shape
dff.columns = ['Registration No.','Name','Engine No.','Manufacturing Date','Registration No.','Chasis No.']
# Saving to .csv file.
dff.to_csv('info.csv')